In [1]:
import pandas as pd
from functions import assembleDf, epochG, epochsG
from datetime import datetime, timedelta
import numpy as np
import sys
sys.path.append('..')
from pyglicko2.glicko2_tests import exampleCase
from pyglicko2.glicko2 import Player
import glicko2
import time


In [2]:
matches = pd.read_csv('../Data/matches_glicko2.csv',parse_dates = ['tourney_date'], 
                      infer_datetime_format = True)

Because memory constraints limit the batch size in epochsG, I will need to process incrementally.

In [3]:
# base batches.  can be further broken up by splitBatch
eg_batches_50 = [(s,f) for s,f in zip(range(0, 876_978-50_000,50_000),
                      range(50_000, 876978,50_000))] + [(850_000,876979)]
eg_batches_100 = [(s,f) for s,f in zip(range(0, 876_978-100_000,100_000),
                      range(100_000, 876978,100_000))] + [(800_000,876979)]

ratings_histories = []
players_dict = {}

In [4]:
def ceildiv(a, b):
    return -(a // -b)

In [5]:
def splitBatch(atuple, divisions):
    '''Break up batches into smaller pieces to allow epochsG to run.'''
    batch_size = ceildiv((atuple[1] - atuple[0]),divisions)
    if batch_size < (atuple[1] - atuple[0]):
        batches = [b for b in zip(range(atuple[0],atuple[1]-batch_size,batch_size),
             range(atuple[0]+batch_size,atuple[1],batch_size))]
        extra = [(batches[-1][1],atuple[1])]
        return batches + extra
    else:
        return atuple

In [6]:
%%time
for batch in eg_batches_50[0:1]:
    players_dict,rh = epochsG(matches[batch[0]:batch[1]],players_dict,365)
    ratings_histories += [rh]

CPU times: user 11.4 s, sys: 54.9 ms, total: 11.4 s
Wall time: 11.4 s


In [7]:
%%time

for batch in eg_batches_50[1:2]:
    players_dict,rh = epochsG(matches[batch[0]:batch[1]],players_dict,365)
    ratings_histories += [rh]

CPU times: user 4.28 s, sys: 21.6 ms, total: 4.3 s
Wall time: 4.28 s


In [8]:
%%time

for batch in eg_batches_50[2:3]:
    players_dict,rh = epochsG(matches[batch[0]:batch[1]],players_dict,365)
    ratings_histories += [rh]

CPU times: user 4.4 s, sys: 19.2 ms, total: 4.42 s
Wall time: 4.41 s


In [9]:
eg_batches_50[3]

(150000, 200000)

In [10]:
b = splitBatch(eg_batches_50[3],2); b0 = b[0]; b1 = b[1]
b

[(150000, 175000), (175000, 200000)]

In [11]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 2.1 s, sys: 6.1 ms, total: 2.1 s
Wall time: 2.1 s


In [12]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 2.09 s, sys: 6.91 ms, total: 2.09 s
Wall time: 2.09 s


In [13]:
splitBatch(eg_batches_50[4],3)

[(200000, 216667), (216667, 233334), (233334, 250000)]

In [14]:
b = splitBatch(eg_batches_50[4],4); 
b0 = b[0]; b1 = b[1]; b2 = b[2]; b3=b[3]
b

[(200000, 212500), (212500, 225000), (225000, 237500), (237500, 250000)]

In [15]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.14 s, sys: 6.02 ms, total: 1.15 s
Wall time: 1.14 s


In [16]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.07 s, sys: 5.73 ms, total: 1.07 s
Wall time: 1.07 s


In [17]:
%%time
players_dict,rh = epochsG(matches[b2[0]:b2[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.12 s, sys: 4.29 ms, total: 1.12 s
Wall time: 1.12 s


In [18]:
%%time
players_dict,rh = epochsG(matches[b3[0]:b3[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.19 s, sys: 7.95 ms, total: 1.2 s
Wall time: 1.19 s


In [19]:
%%time
players_dict,rh = epochsG(matches[eg_batches_50[5][0]:eg_batches_50[5][1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 3.97 s, sys: 7.83 ms, total: 3.98 s
Wall time: 3.98 s


%%time
players_dict,rh = epochsG(matches[eg_batches_50[6][0]:eg_batches_50[6][1]],players_dict,365)
ratings_histories += [rh]

In [20]:
b = splitBatch(eg_batches_50[6],8); 
b0 = b[0]; b1 = b[1]; b2 = b[2]; b3=b[3];
b4 = b[0]; b5 = b[1]; b6 = b[2]; b7=b[3]
b

[(300000, 306250),
 (306250, 312500),
 (312500, 318750),
 (318750, 325000),
 (325000, 331250),
 (331250, 337500),
 (337500, 343750),
 (343750, 350000)]

In [21]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 873 ms, sys: 5.61 ms, total: 879 ms
Wall time: 877 ms


In [22]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 906 ms, sys: 7.07 ms, total: 913 ms
Wall time: 911 ms


In [23]:
%%time
players_dict,rh = epochsG(matches[b2[0]:b2[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 772 ms, sys: 2.82 ms, total: 775 ms
Wall time: 774 ms


In [24]:
%%time
players_dict,rh = epochsG(matches[b3[0]:b3[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 885 ms, sys: 3.55 ms, total: 889 ms
Wall time: 889 ms


In [25]:
%%time
players_dict,rh = epochsG(matches[b4[0]:b4[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 875 ms, sys: 11.6 ms, total: 887 ms
Wall time: 877 ms


In [26]:
%%time
players_dict,rh = epochsG(matches[b5[0]:b5[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 930 ms, sys: 10.4 ms, total: 941 ms
Wall time: 932 ms


In [27]:
%%time
players_dict,rh = epochsG(matches[b6[0]:b6[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 831 ms, sys: 13.7 ms, total: 845 ms
Wall time: 832 ms


In [28]:
%%time
players_dict,rh = epochsG(matches[b7[0]:b7[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 879 ms, sys: 7.31 ms, total: 887 ms
Wall time: 906 ms


%%time

players_dict,rh = epochsG(matches[eg_batches_50[7][0]:eg_batches_50[7][1]],players_dict,365)
ratings_histories += [rh]

In [29]:
b = splitBatch(eg_batches_50[7],5); 
b0 = b[0]; b1 = b[1]; b2 = b[2]; b3 = b[3]; b4 = b[4];


In [ ]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[b2[0]:b2[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[b2[0]:b2[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[b3[0]:b3[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[b4[0]:b4[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
b = splitBatch(eg_batches_50[8],2); b0 = b[0]; b1 = b[1]
b

In [ ]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
b = splitBatch(eg_batches_50[9],3); b0 = b[0]; b1 = b[1]; 
b2 = b[2]
b

In [ ]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
b = splitBatch(eg_batches_50[10],2); b0 = b[0]; b1 = b[1]
b

In [ ]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
b = splitBatch(eg_batches_50[11],4); b0 = b[0]; b1 = b[1];
b2 = b[0]; b3 = b[1];
b

In [ ]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[eg_batches_50[11][0]:eg_batches_50[11][1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
b = splitBatch(eg_batches_50[12],2); b0 = b[0]; b1 = b[1]
b

In [ ]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[eg_batches_50[13][0]:eg_batches_50[13][1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
b = splitBatch(eg_batches_50[14],2); b0 = b[0]; b1 = b[1]
b

In [ ]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[eg_batches_50[15][0]:eg_batches_50[15][1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[eg_batches_50[16][0]:eg_batches_50[16][1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[eg_batches_50[17][0]:eg_batches_50[17][1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
rh